In [3]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [7]:
data_text.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [9]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/matthewhwang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [16]:
stemmer = SnowballStemmer("english")
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [17]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [18]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [19]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [20]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (484, 1), (4015, 1)]

In [21]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4015 ("dampen") appears 1 time.


In [22]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5893154583024485),
 (1, 0.3892866165028569),
 (2, 0.49651921997736453),
 (3, 0.5046106271280878)]


In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [24]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.021*"warn" + 0.021*"market" + 0.018*"hospit" + 0.016*"health" + 0.015*"share" + 0.015*"school" + 0.014*"minist" + 0.014*"break" + 0.013*"worker" + 0.013*"student"
Topic: 1 
Words: 0.026*"australian" + 0.026*"adelaid" + 0.020*"test" + 0.017*"tasmanian" + 0.017*"win" + 0.016*"open" + 0.014*"melbourn" + 0.011*"vote" + 0.011*"darwin" + 0.009*"parti"
Topic: 2 
Words: 0.024*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.012*"show" + 0.012*"price" + 0.011*"search"
Topic: 3 
Words: 0.030*"govern" + 0.022*"south" + 0.020*"north" + 0.017*"countri" + 0.016*"fund" + 0.015*"rural" + 0.014*"hour" + 0.013*"west" + 0.013*"indigen" + 0.013*"australia"
Topic: 4 
Words: 0.021*"die" + 0.020*"live" + 0.018*"donald" + 0.017*"coast" + 0.015*"gold" + 0.014*"lose" + 0.013*"crash" + 0.013*"park" + 0.012*"beat" + 0.012*"royal"
Topic: 5 
Words: 0.028*"elect" + 0.021*"sydney" + 0.018*"say" + 0.014*"labor" + 0.014*"time" + 0.012*"cal

In [25]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.016*"crash" + 0.013*"polic" + 0.013*"miss" + 0.012*"die" + 0.010*"search" + 0.010*"woman" + 0.009*"death" + 0.008*"bodi" + 0.007*"shoot" + 0.007*"fatal"
Topic: 1 Word: 0.009*"energi" + 0.008*"ash" + 0.008*"decemb" + 0.006*"terror" + 0.006*"obama" + 0.005*"travel" + 0.005*"philippin" + 0.005*"refus" + 0.005*"renew" + 0.005*"robert"
Topic: 2 Word: 0.011*"world" + 0.010*"final" + 0.010*"drum" + 0.009*"australia" + 0.009*"leagu" + 0.007*"win" + 0.006*"open" + 0.006*"hobart" + 0.006*"climat" + 0.006*"rugbi"
Topic: 3 Word: 0.019*"charg" + 0.017*"murder" + 0.015*"court" + 0.011*"alleg" + 0.010*"assault" + 0.010*"jail" + 0.010*"sentenc" + 0.009*"polic" + 0.009*"guilti" + 0.009*"accus"
Topic: 4 Word: 0.009*"marriag" + 0.009*"cattl" + 0.006*"senat" + 0.006*"wrap" + 0.006*"know" + 0.005*"polit" + 0.005*"dump" + 0.005*"australia" + 0.005*"sale" + 0.004*"say"
Topic: 5 Word: 0.008*"kill" + 0.007*"truck" + 0.006*"islam" + 0.006*"driver" + 0.005*"michael" + 0.005*"pacif" + 0.005*"syri

In [26]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [27]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5313934087753296	 
Topic: 0.027*"kill" + 0.025*"home" + 0.019*"attack" + 0.017*"turnbul" + 0.014*"protest" + 0.014*"road" + 0.014*"driver" + 0.013*"farm" + 0.012*"polic" + 0.012*"forc"

Score: 0.30855828523635864	 
Topic: 0.030*"govern" + 0.022*"south" + 0.020*"north" + 0.017*"countri" + 0.016*"fund" + 0.015*"rural" + 0.014*"hour" + 0.013*"west" + 0.013*"indigen" + 0.013*"australia"

Score: 0.020007742568850517	 
Topic: 0.024*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.012*"show" + 0.012*"price" + 0.011*"search"

Score: 0.020007017999887466	 
Topic: 0.021*"warn" + 0.021*"market" + 0.018*"hospit" + 0.016*"health" + 0.015*"share" + 0.015*"school" + 0.014*"minist" + 0.014*"break" + 0.013*"worker" + 0.013*"student"

Score: 0.020006069913506508	 
Topic: 0.038*"polic" + 0.028*"charg" + 0.026*"court" + 0.020*"murder" + 0.017*"face" + 0.015*"woman" + 0.015*"alleg" + 0.014*"interview" + 0.014*"jail" + 0.014*"death"

Sc

In [28]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))



Score: 0.47938376665115356	 
Topic: 0.019*"countri" + 0.018*"trump" + 0.017*"hour" + 0.012*"news" + 0.009*"podcast" + 0.008*"nation" + 0.008*"rural" + 0.008*"elect" + 0.007*"busi" + 0.006*"council"

Score: 0.3605324625968933	 
Topic: 0.014*"market" + 0.010*"turnbul" + 0.010*"share" + 0.009*"rise" + 0.008*"price" + 0.008*"christma" + 0.008*"grandstand" + 0.007*"australian" + 0.006*"fall" + 0.006*"dollar"

Score: 0.020019669085741043	 
Topic: 0.020*"interview" + 0.011*"donald" + 0.010*"weather" + 0.010*"north" + 0.008*"south" + 0.008*"korea" + 0.007*"octob" + 0.007*"wednesday" + 0.006*"thursday" + 0.006*"toni"

Score: 0.020011648535728455	 
Topic: 0.016*"crash" + 0.013*"polic" + 0.013*"miss" + 0.012*"die" + 0.010*"search" + 0.010*"woman" + 0.009*"death" + 0.008*"bodi" + 0.007*"shoot" + 0.007*"fatal"

Score: 0.020010173320770264	 
Topic: 0.010*"rural" + 0.010*"health" + 0.007*"royal" + 0.007*"govern" + 0.007*"farm" + 0.007*"sport" + 0.006*"fund" + 0.006*"commiss" + 0.006*"indigen" + 0.00

In [29]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.32354652881622314	 Topic: 0.033*"trump" + 0.024*"world" + 0.024*"australia" + 0.016*"final" + 0.015*"china"
Score: 0.21015584468841553	 Topic: 0.021*"warn" + 0.021*"market" + 0.018*"hospit" + 0.016*"health" + 0.015*"share"
Score: 0.18332955241203308	 Topic: 0.038*"polic" + 0.028*"charg" + 0.026*"court" + 0.020*"murder" + 0.017*"face"
Score: 0.18286602199077606	 Topic: 0.028*"elect" + 0.021*"sydney" + 0.018*"say" + 0.014*"labor" + 0.014*"time"
Score: 0.016689125448465347	 Topic: 0.030*"govern" + 0.022*"south" + 0.020*"north" + 0.017*"countri" + 0.016*"fund"
Score: 0.016682587563991547	 Topic: 0.026*"australian" + 0.026*"adelaid" + 0.020*"test" + 0.017*"tasmanian" + 0.017*"win"
Score: 0.016682587563991547	 Topic: 0.024*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern"
Score: 0.016682587563991547	 Topic: 0.021*"die" + 0.020*"live" + 0.018*"donald" + 0.017*"coast" + 0.015*"gold"
Score: 0.016682587563991547	 Topic: 0.037*"year" + 0.030*"queensland" + 0.02